# Periodic padding in tensorflow

In [1]:
import numpy as np
import tensorflow as tf
tf.enable_eager_execution()
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
%matplotlib notebook

We're interested in implementing a convolutional neural net which acts on periodic data.
Unfortunately, periodic padding doesn't seem to be natively available in tensorflow or pytorch.
This notebook will explore how to produce periodically-padded tensorflow convolutions with neutral, up, and down striding.

## Discrete periodic convolutions

We will define the discrete convolution $y = k * x$, with a kernel of size P, centered on index $d$, and acting on data of size N, in index notation as

$$
y_i = \sum_{p=0}^{P-1} k_{p} x_{(i+p-d) (\mathrm{mod} N)}
$$

where the indeces are taken to be modulo N via the underlying periodicity.

This can be rewritten as a matrix multiplication $y = K \cdot x$, or 

$$y_i = \sum_{j=0}^{N-1} K_{ij} x_j$$

where $K_{ij} = k_{(j-i+d) (\mathrm{mod} N)}$ and $k_p = 0$ for $p \ge P$.

Let's look at a quick numerical example:

In [65]:
# Set sizes and build simple kernel
N = 8
P = 3
d = 1
k = np.arange(1, P+1)
print('Kernel:', k)

# Build K matrix (slow algorithm)
K = np.zeros((N, N), dtype=int)
for i in range(N):
    for j in range(N):
        p = (j - i + d) % N
        if p < P:
            K[i, j] = k[p]
            
print('K matrix:')
print(K)

Kernel: [1 2 3]
K matrix:
[[2 3 0 0 0 0 0 1]
 [1 2 3 0 0 0 0 0]
 [0 1 2 3 0 0 0 0]
 [0 0 1 2 3 0 0 0]
 [0 0 0 1 2 3 0 0]
 [0 0 0 0 1 2 3 0]
 [0 0 0 0 0 1 2 3]
 [3 0 0 0 0 0 1 2]]


## Padding to achieve periodicity

Since the tensorflow convolutions do not implement the periodic wrapping we need (i.e. indexing in modular arithmetic), we'll implement the same via periodically padding our input to size $N + P - 1$.

We'll represent the padding operator as the linear operator $z = e(x, P-1)$, which in index notation is simply $
z_i = x_{(i-d) (mod N)}$.
This can again be rewritten as a matrix multiplication $z = E \cdot x$ where $E$ is a a matrix with shape $(N+P-1, N)$ and $E_{ij} = \delta_{(i-d) (\mathrm{mod} N), j}$.

Let's take a look at a numerical example:

In [66]:
# Build extension matrix (slow algorithm)
E = np.zeros((N+P-1, N), dtype=int)
for i in range(N+P-1):
    for j in range(N):
        if (i-d) % N == j:
            E[i, j] = 1
            
print('E matrix:')
print(E)

# Apple to an example
x = np.arange(1, N+1)
print('x: \n ', x)
z = E @ x
print('z = G @ x: \n ', z)

E matrix:
[[0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1]
 [1 0 0 0 0 0 0 0]]
x: 
  [1 2 3 4 5 6 7 8]
z = G @ x: 
  [8 1 2 3 4 5 6 7 8 1]


We can now write a new convolution matrix which acts with "valid" padding on $z$ as $y = G \cdot z$ where $G$ has shape $(N, N+P-1)$ and $G_{ij} = k_{j-i}$.

We can verify the correctness of this formulion analytically:
$$
\begin{align}
y_i &= \sum_{j=0}^{N+P-2} G_{ij} z_j \\
&= \sum_{j=0}^{N+P-2} k_{j-i} x_{(j-d) (\mathrm{mod} N)} \\
&= \sum_{p=-i}^{N+P-2-i} k_{p} x_{(i+p-d) (\mathrm{mod} N)} \\
&= \sum_{p=0}^{P-1} k_{p} x_{(i+p-d) (\mathrm{mod} N)} \\
\end{align}
$$

Let's check numerically:

In [67]:
# Build extended convolution matrix (slow algorithm)
G = np.zeros((N, N+P-1), dtype=int)
for i in range(N):
    for j in range(N+P-1):
        p = j - i
        if 0 <= p < P:
            G[i, j] = k[p]
            
print('G matrix:')
print(G)

# Test reformulation
y0 = K @ x
print('y = K @ x: \n ', y0)
y1 = G @ z
print('y = G @ E @ x: \n ', y1)
print('  Same:', np.allclose(y0, y1))

G matrix:
[[1 2 3 0 0 0 0 0 0 0]
 [0 1 2 3 0 0 0 0 0 0]
 [0 0 1 2 3 0 0 0 0 0]
 [0 0 0 1 2 3 0 0 0 0]
 [0 0 0 0 1 2 3 0 0 0]
 [0 0 0 0 0 1 2 3 0 0]
 [0 0 0 0 0 0 1 2 3 0]
 [0 0 0 0 0 0 0 1 2 3]]
y = K @ x: 
  [16 14 20 26 32 38 44 26]
y = G @ E @ x: 
  [16 14 20 26 32 38 44 26]
  Same: True


Now let's give it a try in tensorflow:

In [68]:
# Reshape input as (batch, *size, channels)
z_tens = tf.convert_to_tensor(z[None, None, :, None].astype(float))
# Reshape kernel as (*size, channels, features)
k_tens = tf.convert_to_tensor(k[None, :, None, None].astype(float))

y2_tens = keras.backend.conv2d(z_tens, k_tens, padding='valid')
y2 = y2_tens.numpy().flatten()

print('y = (tf conv): \n ', y2)
print('  Same:', np.allclose(y0, y2))

y = (tf conv): 
  [16. 14. 20. 26. 32. 38. 44. 26.]
  Same: True


## Downsampling convolutions

Downsampling via striding is equivalent to simply striding the output, or striding the rows of $K$ or $G$, which we can write as $S \cdot K = S \cdot G \cdot E$.
The tensorflow convolutions should do the same, even when acting on the padded inputs, since they've been padded just enough to properly convolve the original input.

Let's check this numerically:

In [69]:
# Set stride size
s = 4

# Run strided convolution
y3_tens = keras.backend.conv2d(z_tens, k_tens, padding='valid', strides=(1,s))
y3 = y3_tens.numpy().flatten()

print('y = (strided tf conv): \n ', y3)
print('  Same:', np.allclose(y0[::s], y3))

y = (strided tf conv): 
  [16. 32.]
  Same: True


## Upsampling convolutions

Upsampling via striding is equivalent to transposing the strided $K$, i.e. $(S \cdot K)^T = E^T \cdot (S \cdot G)^T$.
The second operator $(S \cdot G)^T$ is implemented via the strided transpose convolutions in tensorflow, so we can produce the full convolution by applying $E.T$ to the output of this layer.

Let's check this numerically:

In [70]:
# Produce input data
M = (N + s - 1) // s
y = np.arange(1, M+1)

# Direct up-convolutions
x0 = K[::s].T @ y
print('x = K[::s].T @ y: \n ', x0)
x1 = E.T @ G[::s].T @ y
print('x = E.T @ G[::s].T @ y: \n ', x1)
print('  Same:', np.allclose(x0, x1))

# Reshape input as (batch, *size, channels)
y_tens = tf.convert_to_tensor(y[None, None, :, None].astype(float))

# Run strided up-convolution
output_shape = (z_tens.shape)
x2_tens = keras.backend.conv2d_transpose(y_tens, k_tens, output_shape, padding='valid', strides=(1,s))
x2 = E.T @ x2_tens.numpy().flatten()

print('x = E.T @ (tf conv trans): \n ', x2)
print('  Same:', np.allclose(x0, x2))

x = K[::s].T @ y: 
  [2 3 0 2 4 6 0 1]
x = E.T @ G[::s].T @ y: 
  [2 3 0 2 4 6 0 1]
  Same: True
x = E.T @ (tf conv trans): 
  [2. 3. 0. 2. 4. 6. 0. 1.]
  Same: True


## Conclusions

We have seen that we simply need to have implementations of the forward and transposed padding/extension operators $E$ and $E^T$ to produce periodic convolutions with tensorflow.
These two operators need to be fast and differentiable to be usable in production.

## Custom implementations

Now let's also check our custom tensorflow implementations for correctness.

In [71]:
import unet

### Equisampling

In [72]:
# Reshape input as (batch, *size, channels)
x3_tens = tf.convert_to_tensor(x[None, None, None, :, None].astype(float))
# Reshape kernel as (*size, channels, features)
k3_tens = tf.convert_to_tensor(k[None, None, :, None, None].astype(float))

filters = 1
kernel_size = (1, 1, P)
kernel_center = (0, 0, d)
input_shape = (1, 1, 1, N, 1)
dtype = np.float64
weights = [k3_tens, np.array([0])]

model = unet.PeriodicConv3D(filters, kernel_size, kernel_center)
model(tf.zeros(input_shape, dtype=dtype))
model.set_weights(weights)
y4 = model(x3_tens).numpy().flatten()

print('y = (tf conv): \n ', y4)
print('  Same:', np.allclose(y0, y4))

y = (tf conv): 
  [16. 14. 20. 26. 32. 38. 44. 26.]
  Same: True


### Downsampling

In [89]:
strides = (1, 1, s)

model = unet.PeriodicConv3D(filters, kernel_size, kernel_center, strides=strides)
model(tf.zeros(input_shape, dtype=dtype))
model.set_weights(weights)
y5 = model(x3_tens).numpy().flatten()

print('y = (strided tf conv): \n ', y5)
print('  Same:', np.allclose(y0[::s], y5))

y = (strided tf conv): 
  [16. 32.]
  Same: True


### Upsampling

In [95]:
# Reshape input as (batch, *size, channels)
y3_tens = tf.convert_to_tensor(y[None, None, None, :, None].astype(float))

#model = unet.PeriodicConv3DTranspose(filters, kernel_size, kernel_center, strides=strides)
model = keras.layers.Conv2DTranspose(filters, kernel_size[1:], strides=strides[1:], padding='valid')
model(tf.zeros(y3_tens.shape[1:], dtype=dtype))
model.set_weights([k3_tens[0], np.array([0])])
x3 = model(y3_tens[0]).numpy().flatten()

print('x = E.T @ (tf conv trans): \n ', x3)
#print('  Same:', np.allclose(x0, x3))
print(x0)

x = E.T @ (tf conv trans): 
  [1. 2. 3. 0. 2. 4. 6. 0.]
[2 3 0 2 4 6 0 1]
